In [ ]:
"""
Author: Roel Faber
Date: 27/04/2019

Analyzing tweets of Dutch MP's
""" 

In [87]:
import os
import json
from pprint import pprint
from collections import Counter
import pandas as pd
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import twitter
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

import setup

In [89]:
api = twitter.Api(consumer_key = setup.consumer_key,
                  consumer_secret = setup.consumer_secret,
                  access_token_key = setup.access_token_key,
                  access_token_secret = setup.access_token_secret)

In [ ]:
def gettwitter():
    d = {}
    browser = webdriver.Chrome()
    browser.get('https://www.tweedekamer.nl/kamerleden_en_commissies/alle_kamerleden')
    data = requests.get(browser.current_url)
    soup = BeautifulSoup(data.text,'html.parser')
    soup.findAll('a', {'class': 'member__name'})[0].text
    members = soup.findAll('a', {'class': 'member__name'})
    memberlinksnames = []
    for link in members:
        try:
            member = link['href'], link.text
            memberlinksnames.append(member)
        except KeyError:
            print(link.text)
    memberlinksnames = memberlinksnames[:25]
    for member in memberlinksnames:
        link, name = member
        browser.get(f'https://www.tweedekamer.nl{link}')
        data = requests.get(browser.current_url)
        soup = BeautifulSoup(data.text,'html.parser')
        try:
            twitter = soup.findAll('a', {'class':'___twitter'})[1].text
        except IndexError:
            twitter = 'no twitter'
        partij = browser.current_url.split('-')[-1].upper()
        d[name] = twitter, partij
    return d

In [92]:
def missingtwitters(d, api):
    l = []
    for key, value in d.items():
        if value[0] == 'no twitter':
            results = api.GetUsersSearch(term = key)
            try:
                twit = '@' + results[0].screen_name
                d[key] = twit, value[1]
            except:
                next
    return d

In [4]:
def gettimeline(screenname,api):
    statuses = api.GetUserTimeline(screen_name = screenname,count=200)
    with open('Data/' + screenname[1:]+'.txt','a') as f:
        l = []
        for line in statuses:
            f.write(str(line))
            f.write('\n')
            text = line.text
            for word in line.text.split(' '):
                if word.startswith('@'):
                    wordfix = word.translate({ord(c): None for c in [':','.']})
                    l.append(wordfix)
    return l

In [9]:
def main():
    api = twitter.Api(consumer_key = 'Stt06KmZ5KWqjDjURb1pyP5A4',
                 consumer_secret = 'mYIFsaWqauD5rOY7gQNz2aUZVemvFwgPQjiXl7KYImoFZ1GYcj',
                 access_token_key = '974647689915838465-uvKGIoH8ypb6xTbPnV7ddRullyC0UIo',
                 access_token_secret = 'VHI6LG7rkkAaO7rRLoFC5PNxPJ4h5M0wPaCUQWtu8wRqr')
    d = gettwitter()
    d = missingtwitters(d,api)
    d['Chantal Nijkerken-de Haan'] = '@cnijkerken','vvd'
    d['Sybrand van Haersma Buma'] = '@sybrandbuma', 'cda'
    serieslist = []
    listlist = []
    partijdict = {}
    for key, value in d.items():
        if value[0] != 'no twitter':
            try:
                l = gettimeline(value[0],api)
            except:
                results = api.GetUsersSearch(term = key)
                try:
                    twit = '@' + results[0].screen_name
                    d[key] = twit, value[1]
                    l = gettimeline(twit,api)
                except:
                    print(key)
                    next
            listlist.append(l)
            series = pd.Series(Counter(l),name = key)
            serieslist.append(series)
            partijdict[key] = value[1]
    twithandles = []
    for value in d.values():
        twithandles.append(value[0])
    df = pd.DataFrame(serieslist)
    df = df[df.columns.intersection(twithandles)]
    df = df.fillna(value=0)
    partijseries = pd.Series(partijdict, name = 'partij')
    df = pd.concat([partijseries,df],axis=1)
    df.drop('0',axis=1,inplace=True)
    df
    FG = nx.convert_matrix.from_pandas_adjacency(df)
    return (FG,df)
    
FG,df = main()
    

Lilianne Ploumen


NameError: name 'partijseries' is not defined

In [6]:
d = gettwitter()
d = missingtwitters(d,api)
d['Chantal Nijkerken-de Haan'] = '@cnijkerken','vvd'
d['Sybrand van Haersma Buma'] = '@sybrandbuma', 'cda'

In [7]:
listlist = []
serieslist = []
partijdict = {}
for key, value in d.items():
    if value[0] != 'no twitter':
        try:
            l = gettimeline(value[0],api)
        except:
            results = api.GetUsersSearch(term = key)
            try:
                twit = '@' + results[0].screen_name
                d[key] = twit, value[1]
                l = gettimeline(twit,api)
            except:
                print(key)
                next
        listlist.append(l)
        series = pd.Series(Counter(l),name = key)
        serieslist.append(series)
        partijdict[key] = value[1]
twithandles = []
for value in d.values():
    twithandles.append(value[0])

Lilianne Ploumen


NetworkXError: ('Adjacency matrix is not square.', 'nx,ny=(148, 139)')

In [26]:
df = pd.DataFrame(serieslist)
df = df[df.columns.intersection(twithandles)]
df = df.fillna(value=0)
partijseries = pd.Series(partijdict, name = 'partij')
df = pd.concat([partijseries,df],axis=1)
df

,partij,@APechtold,@AgnesMulderCDA,@AnneKuik,@AntjeDiertens,@Antoinettelaan,@AntoonKanis,@ArneWeverling,@AvandenBoschVVD,@BisschopRoelof,...,@rvanaalst,@selcukozturknl,@suzanne_GL,@svanweyenberg,@swsjoerdsma,@sybrandbuma,@thierrybaudet,@tunahankuzu,@wimjanrenkema,@wmoorlag
Achraf Bouali,d66,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0
Agnes Mulder,cda,0.0,29.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
Albert van den Bosch,vvd,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
Alexander Kops,pvv,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alexander Pechtold,d66,23.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,6.0,0.0,2.0,0.0,0.0,0.0
André Bosman,vvd,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
Anne Kuik,cda,0.0,12.0,19.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Anne Mulder,vvd,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
Antje Diertens,d66,3.0,1.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0
Antoinette Laan-Geselschap,vvd,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
indexlist = []
for i in df.index:
    indexlist.append(d[i][0])
df.index = indexlist

In [28]:
for index, row in df.iterrows():
    if index not in df.columns:
        df[index] = 0

In [29]:
df.sort_index(inplace=True)
df.sort_index(axis=1,inplace=True)

In [32]:
df.to_csv('TK/Data/csv/network.csv')
df.to_excel('TK/Data/csv/network.xlsx')

In [17]:
FG = nx.convert_matrix.from_pandas_adjacency(df)

In [37]:
df['partij']

@@leendertdelange       vvd
@@poortvli              cda
@@sneller               d66
@APechtold              d66
@AgnesMulderCDA         cda
@AnneKuik               cda
@AntjeDiertens          d66
@Antoinettelaan         vvd
@AntoonKanis             sp
@ArneWeverling          vvd
@AvandenBoschVVD        vvd
@BisschopRoelof         sgp
@BramvanOjikGL           gl
@CemLacin                sp
@ChrisvanDam            cda
@CorinneEllemeet         gl
@CvanBrenk           50plus
@Danai_PVV              pvv
@DennisWiersma          vvd
@DilanYesilgoz          vvd
@EmielVanDijkPVV        pvv
@ErikRonnes             cda
@EvertJanSlootwe        cda
@F_azarkan             denk
@FemkeMerel            pvdd
@FleurAgemaPVV          pvv
@FrankFutselaar          sp
@GidiMarkuszower        pvv
@GraafdeMachiel         pvv
@GroothuizenD66         d66
                      ...  
@jesseklaver             gl
@jpaternotte            d66
@keesvdstaaij           sgp
@khadijaArib           pvda
@kirstenvdhul       

In [33]:
## Here Gephi is used to change the matrix into edges and nodes

In [64]:
nodes = pd.read_csv('TK/Data/csv/nodes.csv',index_col='Id')

In [62]:
nodes2 = pd.concat([nodes,df['partij']],axis=1)

In [66]:
nodes2.to_csv('TK/Data/csv/nodes2.csv')

In [67]:
nodes2

,Label,timeset,partij
@@leendertdelange,@@leendertdelange,NaN,vvd
@@poortvli,@@poortvli,NaN,cda
@@sneller,@@sneller,NaN,d66
@APechtold,@APechtold,NaN,d66
@AgnesMulderCDA,@AgnesMulderCDA,NaN,cda
@AnneKuik,@AnneKuik,NaN,cda
@AntjeDiertens,@AntjeDiertens,NaN,d66
@Antoinettelaan,@Antoinettelaan,NaN,vvd
@AntoonKanis,@AntoonKanis,NaN,sp
@ArneWeverling,@ArneWeverling,NaN,vvd
